In [1]:
#initialize a network
from random import seed, random
def init_net(n_in, n_h, n_out):
    net = list()
    hidden = [{'weights':[random() for i in range(n_in+1)]} for i in range(n_h)]
    net.append(hidden)
    out = [{'weights':[random() for i in range(n_h+1)]} for i in range(n_out)]
    net.append(out)
    return net

In [2]:
#test 
net = init_net(2,1,2)
for layer in net:
    print(layer)

[{'weights': [0.5859471434981559, 0.16731979887911075, 0.05065492000086447]}]
[{'weights': [0.20228693015527477, 0.3660296229628127]}, {'weights': [0.263287319778291, 0.18189775591004853]}]


In [3]:
#defining the forward propagation

import math

def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

def sigmoid(activation):
    return 1.0 / (1.0 + math.exp(-activation))

def forwardprop(net, row):
    inputs = row
    for layer in net:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = sigmoid(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

In [4]:
#test 
row = [1, 0, None]
out = forwardprop(net, row)
print(out)

[0.6220649682488409, 0.5876075140707483]


In [5]:
#the backprop step

def grad(output):
    return output * (1 - output)

def backprop(net, actual):
    for i in reversed(range(len(net))):
        layer = net[i]
        errors = list()
        if i != len(net) - 1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in net[i+1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                    errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(actual[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * grad(neuron['output'])

In [6]:
net = net
actual = [1, 0]
backprop(net, actual)
for layer in net:
    print(layer)

[{'weights': [0.5859471434981559, 0.16731979887911075, 0.05065492000086447], 'output': 0.6539849490406724, 'delta': 0.004067247597055389}]
[{'weights': [0.20228693015527477, 0.3660296229628127], 'output': 0.6220649682488409, 'delta': 0.08885258020835561}, {'weights': [0.263287319778291, 0.18189775591004853], 'output': 0.5876075140707483, 'delta': -0.14239194588249382}]


In [7]:
#update weights with error

def update(net, row, lr):
    for i in range(len(net)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in net[i-1]]
        for neuron in net[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += lr * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += lr * neuron['delta']

In [8]:
#train the network
def train_net(net, train, lr, n_epochs, n_out):
    for epoch in range(n_epochs):
        sum_err = 0
        for row in train:
            outputs = forwardprop(net, row)
            actual = [0 for i in range(n_out)]
            actual[row[-1]] = 1
            sum_err += sum([(actual[i] - outputs[i])**2 for i in range(len(actual))])
            backprop(net, actual)
            update(net, row, lr)
        print('>epoch=%d, lr=%.3f, error=%.3f' %(epoch, lr, sum_err))

In [9]:
dataset = [[2.7810836,2.550537003,0],
    [1.465489372,2.362125076,0],
    [3.396561688,4.400293529,0],
    [1.38807019,1.850220317,0],
    [3.06407232,3.005305973,0],
    [7.627531214,2.759262235,1],
    [5.332441248,2.088626775,1],
    [6.922596716,1.77106367,1],
    [8.675418651,-0.242068655,1],
    [7.673756466,3.508563011,1]]
n_in = len(dataset[0]) - 1
n_out = len(set([row[-1] for row in dataset]))
net = init_net(n_in, 2, n_out)
train_net(net, dataset, 0.5, 25, n_out)
for layer in net:
    print(layer)

>epoch=0, lr=0.500, error=6.557
>epoch=1, lr=0.500, error=5.788
>epoch=2, lr=0.500, error=5.471
>epoch=3, lr=0.500, error=5.247
>epoch=4, lr=0.500, error=4.959
>epoch=5, lr=0.500, error=4.559
>epoch=6, lr=0.500, error=4.063
>epoch=7, lr=0.500, error=3.644
>epoch=8, lr=0.500, error=3.265
>epoch=9, lr=0.500, error=2.921
>epoch=10, lr=0.500, error=2.614
>epoch=11, lr=0.500, error=2.342
>epoch=12, lr=0.500, error=2.101
>epoch=13, lr=0.500, error=1.890
>epoch=14, lr=0.500, error=1.704
>epoch=15, lr=0.500, error=1.542
>epoch=16, lr=0.500, error=1.401
>epoch=17, lr=0.500, error=1.277
>epoch=18, lr=0.500, error=1.168
>epoch=19, lr=0.500, error=1.072
>epoch=20, lr=0.500, error=0.988
>epoch=21, lr=0.500, error=0.914
>epoch=22, lr=0.500, error=0.848
>epoch=23, lr=0.500, error=0.790
>epoch=24, lr=0.500, error=0.738
[{'weights': [-1.0688168018974102, 1.1746707843745887, 0.9537546538571334], 'output': 0.046573917451740936, 'delta': -0.0029823446074322218}, {'weights': [-1.2863332733023307, 1.5902666

In [14]:
# Make a prediction with a network
def predict(network, row):
    outputs = forwardprop(net, row)
    return outputs.index(max(outputs))

In [15]:
for row in dataset:
    prediction = predict(net, row)
    print('actual=%d, predicted=%d' % (row[-1], prediction))

actual=0, predicted=0
actual=0, predicted=0
actual=0, predicted=0
actual=0, predicted=0
actual=0, predicted=0
actual=1, predicted=1
actual=1, predicted=1
actual=1, predicted=1
actual=1, predicted=1
actual=1, predicted=1
